In [ ]:
import pymysql
from datetime import datetime, timedelta
import random

# MySQL 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

cursor = connection.cursor()

# 최근 기록을 가져오는 쿼리
query_latest_metrics = """
SELECT 
    user_id, 
    weight AS current_weight,
    fat_percent AS current_fat_percent,
    muscle_mass AS current_muscle_mass,
    height / 100.0 AS height_m -- 키를 m 단위로 변환
FROM user_body_metrics AS ubm
WHERE record_date = (
    SELECT MAX(record_date) 
    FROM user_body_metrics 
    WHERE user_id = ubm.user_id
)
AND user_id BETWEEN 3 AND 32;
"""

cursor.execute(query_latest_metrics)
latest_metrics = cursor.fetchall()

# 목표 생성 및 삽입
insert_goals_query = """
INSERT INTO user_goals (user_id, target_weight, target_fat_per, target_muscle_mass, start_date, end_date)
VALUES (%s, %s, %s, %s, %s, %s);
"""

start_date = datetime.now().date()

for row in latest_metrics:
    user_id, current_weight, current_fat_percent, current_muscle_mass, height_m = row

    # 현재 BMI 계산
    current_bmi = current_weight / (height_m ** 2)

    # 체중 목표 설정 (BMI 기반 + 편차)
    if current_bmi < 16:
        base_weight_change = 5
    elif current_bmi < 18.5:
        base_weight_change = 3
    elif current_bmi > 30:
        base_weight_change = -5
    elif current_bmi > 25:
        base_weight_change = -3
    else:
        base_weight_change = 0
    
    target_weight = current_weight + base_weight_change + random.uniform(-2, 2)  # 무작위 편차 추가

    # 체지방률 목표 설정 (기본 변화량 + 편차)
    if current_fat_percent > 25:
        base_fat_change = -5
    elif current_fat_percent < 10:
        base_fat_change = 3
    else:
        base_fat_change = 0
    
    target_fat_percent = current_fat_percent + base_fat_change + random.uniform(-3, 3)  # 무작위 편차 추가

    # 근육량 목표 설정 (기본 변화량 + 편차)
    target_muscle_mass = current_muscle_mass + 2 + random.uniform(-0.5, 0.5)  # 무작위 편차 추가

    # 목표 기간 설정: 30일 ~ 90일 랜덤
    goal_duration = random.randint(30, 90)
    end_date = start_date + timedelta(days=goal_duration)

    # 목표 삽입
    cursor.execute(insert_goals_query, (
        user_id,
        round(target_weight, 2),
        round(target_fat_percent, 2),
        round(target_muscle_mass, 2),
        start_date,
        end_date
    ))

# 데이터베이스 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("user_goals 테이블에 목표 데이터 삽입 완료.")


In [2]:
import pymysql
import pandas as pd
from datetime import date

# 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

def calculate_age(birthday):
    """생년월일로부터 만 나이 계산"""
    today = date.today()
    age = today.year - birthday.year
    
    # 생일이 아직 오지 않았으면 나이 1 감소
    if (today.month, today.day) < (birthday.month, birthday.day):
        age -= 1
    
    return age

def calculate_base_calories(user):
    """기초대사율(BMR) 계산"""
    current_weight = user['current_weight']
    current_height = user['current_height']
    age = calculate_age(user['birthday'])
    sex = user['sex']
    
    if sex == 'male':
        # 헤리스-베네딕트 공식 (남성)
        bmr = 88.362 + (13.397 * current_weight) + (4.799 * current_height) - (5.677 * age)
        activity_multiplier = 1.55  # 중간 정도 활동적
    else:
        # 헤리스-베네딕트 공식 (여성)
        bmr = 447.593 + (9.247 * current_weight) + (3.098 * current_height) - (4.330 * age)
        activity_multiplier = 1.45  # 중간 정도 활동적
    
    tdee = bmr * activity_multiplier
    
    # 로그 출력
    print(f"User ID: {user['user_id']}, BMR: {bmr:.2f}, TDEE: {tdee:.2f}")
    return tdee

def calculate_macro_goals(user, diet_strategy):
    """개인화된 매크로 영양소 목표 설정"""
    base_calories = calculate_base_calories(user)
    
    # 체중 관리 목표에 따른 칼로리 조정
    if 'weight_loss' in diet_strategy:
        base_calories -= 300  # 체중 감량을 위해 칼로리 감소
    elif 'muscle_gain' in diet_strategy:
        base_calories += 300  # 근육 증가를 위해 칼로리 증가
    
    # 매크로 비율 결정
    if 'muscle_gain' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3
    elif 'weight_loss' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.35, 0.3, 0.35
    else:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3
    
    # 단백질 목표 계산 (체중 기반)
    base_protein = user['current_weight'] * (
        2.2 if 'muscle_gain' in diet_strategy else 
        1.6 if 'weight_loss' in diet_strategy else 
        1.8
    )
    
    # 탄수화물, 지방 목표 계산
    base_carbs = (base_calories * carb_ratio) / 4
    base_fat = (base_calories * fat_ratio) / 9
    
    # 로그 출력
    calculated_total_calories = (base_protein * 4) + (base_carbs * 4) + (base_fat * 9)
    print(f"User ID: {user['user_id']}  "
          f"Protein: {base_protein:.2f}g, Carbs: {base_carbs:.2f}g, Fat: {base_fat:.2f}g, "
          f"Target Calories: {calculated_total_calories:.2f}")

def determine_diet_strategy(user):
    """개인화된 식단 전략 결정"""
    strategies = []
    
    # 체중 변화 전략
    weight_diff = user['target_weight'] - user['current_weight']
    if weight_diff < -2:  # 체중 감량
        strategies.append('weight_loss')
    elif weight_diff > 2:  # 체중 증가
        strategies.append('muscle_gain')
    else:
        strategies.append('maintenance')
    
    # 나이 전략
    age = calculate_age(user['birthday'])
    if age < 25:
        strategies.append('young_high_metabolism')
    elif age > 40:
        strategies.append('slower_metabolism')
    
    # 성별 및 체성분 전략
    if user['sex'] == 'MALE':
        strategies.append('male_muscle_priority')
        if user['current_fat_percent'] > 20:
            strategies.append('fat_reduction')
    else:
        strategies.append('female_lean_priority')
        if user['current_fat_percent'] > 25:
            strategies.append('fat_reduction')
    
    # 근육량 전략
    if user['current_muscle_mass'] < user['target_muscle_mass']:
        strategies.append('muscle_build')
    
    return strategies

try:
    # 데이터베이스에서 데이터 가져오기
    with connection.cursor() as cursor:
        # 통합 쿼리: 사용자 정보, 신체 측정값, 목표 결합
        query = """
        SELECT 
            ui.user_id,
            ui.birthday,
            ui.sex,
            ubm.weight AS current_weight,
            ubm.height AS current_height,
            ubm.fat_percent AS current_fat_percent,
            ubm.muscle_mass AS current_muscle_mass,
            ug.target_weight,
            ug.target_fat_per,
            ug.target_muscle_mass
        FROM user_info ui
        JOIN user_body_metrics ubm ON ui.user_id = ubm.user_id
        JOIN user_goals ug ON ui.user_id = ug.user_id
        WHERE ubm.record_date = (
            SELECT MAX(record_date) 
            FROM user_body_metrics 
            WHERE user_id = ubm.user_id
        )
        """
        
        cursor.execute(query)
        metrics_data = cursor.fetchall()
        metrics_columns = [col[0] for col in cursor.description]
        metrics_df = pd.DataFrame(metrics_data, columns=metrics_columns)

finally:
    connection.close()

# 사용자 ID 입력받아 정보 출력
input_user_id = int(input("Enter User ID: "))
user_data = metrics_df[metrics_df['user_id'] == input_user_id]

if not user_data.empty:
    user = user_data.iloc[0]
    diet_strategy = determine_diet_strategy(user)
    calculate_macro_goals(user, diet_strategy)
else:
    print(f"User ID {input_user_id} not found.")


User ID: 3, BMR: 1440.89, TDEE: 2089.29
User ID: 3  Protein: 111.04g, Carbs: 134.20g, Fat: 69.58g, Target Calories: 1607.20


In [4]:
import pymysql
import pandas as pd
from datetime import date
import requests
from concurrent.futures import ThreadPoolExecutor
from itertools import combinations

# 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

API_KEY = "YYBjxJqR6CexItNR8rwA836gQiaGpQdSeeXCsAh8"

def calculate_age(birthday):
    """생년월일로부터 만 나이 계산"""
    today = date.today()
    age = today.year - birthday.year
    if (today.month, today.day) < (birthday.month, birthday.day):
        age -= 1
    return age

def calculate_base_calories(user):
    """기초대사율(BMR) 계산"""
    current_weight = user['current_weight']
    current_height = user['current_height']
    age = calculate_age(user['birthday'])
    sex = user['sex']

    if sex == 'male':
        bmr = 88.362 + (13.397 * current_weight) + (4.799 * current_height) - (5.677 * age)
        activity_multiplier = 1.55
    else:
        bmr = 447.593 + (9.247 * current_weight) + (3.098 * current_height) - (4.330 * age)
        activity_multiplier = 1.45

    tdee = bmr * activity_multiplier
    return tdee

def calculate_macro_goals(user, diet_strategy):
    """개인화된 매크로 영양소 목표 설정"""
    base_calories = calculate_base_calories(user)
    if 'weight_loss' in diet_strategy:
        base_calories -= 300
    elif 'muscle_gain' in diet_strategy:
        base_calories += 300

    if 'muscle_gain' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3
    elif 'weight_loss' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.35, 0.3, 0.35
    else:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3

    base_protein = user['current_weight'] * (
        2.2 if 'muscle_gain' in diet_strategy else 
        1.6 if 'weight_loss' in diet_strategy else 
        1.8
    )

    base_carbs = (base_calories * carb_ratio) / 4
    base_fat = (base_calories * fat_ratio) / 9

    return base_carbs, base_protein, base_fat

def determine_diet_strategy(user):
    """개인화된 식단 전략 결정"""
    strategies = []
    weight_diff = user['target_weight'] - user['current_weight']
    if weight_diff < -2:
        strategies.append('weight_loss')
    elif weight_diff > 2:
        strategies.append('muscle_gain')
    else:
        strategies.append('maintenance')

    age = calculate_age(user['birthday'])
    if age < 25:
        strategies.append('young_high_metabolism')
    elif age > 40:
        strategies.append('slower_metabolism')

    if user['sex'] == 'MALE':
        strategies.append('male_muscle_priority')
        if user['current_fat_percent'] > 20:
            strategies.append('fat_reduction')
    else:
        strategies.append('female_lean_priority')
        if user['current_fat_percent'] > 25:
            strategies.append('fat_reduction')

    if user['current_muscle_mass'] < user['target_muscle_mass']:
        strategies.append('muscle_build')

    return strategies

def search_foods(query):
    """USDA FoodData Central API를 사용해 음식 데이터 검색"""
    url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        "api_key": API_KEY,
        "query": query,
        "pageSize": 10
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def parallel_search(queries):
    """병렬로 음식 검색"""
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(search_foods, queries))
    return results

def recommend_foods(target_carbs, target_protein, target_fat):
    """목표 탄단지에 맞는 음식 추천"""
    queries = [
        "chicken", "rice", "milk", "beans", "egg", "beef", "pork", "tofu", "salmon", "shrimp",
        "cheese", "yogurt", "oats", "quinoa", "spinach", "broccoli", "potato", "sweet potato", 
        "peanut butter", "almonds", "walnuts"
    ]

    
    # 병렬로 음식 데이터를 가져오기
    results = parallel_search(queries)
    
    all_recommendations = []
    for food_results in results:
        if food_results and "foods" in food_results:
            for food in food_results["foods"]:
                food_name = food.get("description", "Unknown food")
                nutrients = food.get("foodNutrients", [])
                
                # 영양소 정보 추출
                carbs = next((n["value"] for n in nutrients if n.get("nutrientName") == "Carbohydrate, by difference"), 0)
                protein = next((n["value"] for n in nutrients if n.get("nutrientName") == "Protein"), 0)
                fat = next((n["value"] for n in nutrients if n.get("nutrientName") == "Total lipid (fat)"), 0)
                
                # 탄단지 값이 모두 0인 경우 제외
                if carbs == 0 and protein == 0 and fat == 0:
                    continue
                
                # 필요한 양 계산
                ratio_carbs = target_carbs / carbs if carbs > 0 else float('inf')
                ratio_protein = target_protein / protein if protein > 0 else float('inf')
                ratio_fat = target_fat / fat if fat > 0 else float('inf')
                min_ratio = min(ratio_carbs, ratio_protein, ratio_fat)
                
                # 비현실적인 값 필터링
                if min_ratio == float('inf') or min_ratio <= 0:
                    continue
                
                all_recommendations.append({
                    "food_name": food_name,
                    "carbs": carbs,
                    "protein": protein,
                    "fat": fat,
                    "amount_needed": round(min_ratio * 100, 2)  # g 단위로 변환
                })
    
    # 목표 탄단지 비율에 맞게 정렬
    all_recommendations.sort(key=lambda x: abs(target_carbs - x["carbs"]) +
                                        abs(target_protein - x["protein"]) +
                                        abs(target_fat - x["fat"]))
    return all_recommendations

def optimize_food_combination(recommended_foods, target_carbs, target_protein, target_fat, max_foods=3):
    from itertools import combinations
    
    best_combination = None
    min_diff = float('inf')
    
    # 최대 max_foods 개수의 조합 탐색
    for r in range(1, max_foods + 1):
        for combo in combinations(recommended_foods, r):
            # 각 음식의 실제 섭취 가능한 양 계산 (100g 기준)
            total_carbs = sum(food['carbs'] * 1 for food in combo)
            total_protein = sum(food['protein'] * 1 for food in combo)
            total_fat = sum(food['fat'] * 1 for food in combo)
            
            # 목표와의 차이 계산 (가중치 부여)
            carbs_diff = abs(total_carbs - target_carbs)
            protein_diff = abs(total_protein - target_protein)
            fat_diff = abs(total_fat - target_fat)
            
            # 차이에 가중치를 두어 계산
            diff = (carbs_diff * 1.0 + 
                    protein_diff * 1.0 + 
                    fat_diff * 1.0)
            
            if diff < min_diff:
                min_diff = diff
                best_combination = {
                    'foods': combo,
                    'total_carbs': total_carbs,
                    'total_protein': total_protein,
                    'total_fat': total_fat
                }
    
    return best_combination

# 사용자 데이터 가져오기
try:
    with connection.cursor() as cursor:
        query = """
        SELECT 
            ui.user_id,
            ui.birthday,
            ui.sex,
            ubm.weight AS current_weight,
            ubm.height AS current_height,
            ubm.fat_percent AS current_fat_percent,
            ubm.muscle_mass AS current_muscle_mass,
            ug.target_weight,
            ug.target_fat_per,
            ug.target_muscle_mass
        FROM user_info ui
        JOIN user_body_metrics ubm ON ui.user_id = ubm.user_id
        JOIN user_goals ug ON ui.user_id = ug.user_id
        WHERE ubm.record_date = (
            SELECT MAX(record_date) 
            FROM user_body_metrics 
            WHERE user_id = ubm.user_id
        )
        """
        
        cursor.execute(query)
        metrics_data = cursor.fetchall()
        metrics_columns = [col[0] for col in cursor.description]
        metrics_df = pd.DataFrame(metrics_data, columns=metrics_columns)

finally:
    connection.close()

# 사용자 ID 입력받아 정보 출력
input_user_id = 3
user_data = metrics_df[metrics_df['user_id'] == input_user_id]

if not user_data.empty:
    user = user_data.iloc[0]
    diet_strategy = determine_diet_strategy(user)
    target_carbs, target_protein, target_fat = calculate_macro_goals(user, diet_strategy)

    target_carbs /= 3
    target_protein /= 3
    target_fat /= 3

    recommended_foods = recommend_foods(target_carbs, target_protein, target_fat)
    optimal_combo = optimize_food_combination(recommended_foods, target_carbs, target_protein, target_fat)

    if optimal_combo:
        print("\nOptimal Food Combination:")
        print(f"Total Carbs: {optimal_combo['total_carbs']:.2f}g (Target: {target_carbs:.2f}g)")
        print(f"Total Protein: {optimal_combo['total_protein']:.2f}g (Target: {target_protein:.2f}g)")
        print(f"Total Fat: {optimal_combo['total_fat']:.2f}g (Target: {target_fat:.2f}g)")
        print("\nFoods in Combination:")
        for food in optimal_combo['foods']:
            print(f"- {food['food_name']} (Carbs: {food['carbs']}g, Protein: {food['protein']}g, Fat: {food['fat']}g)")
    else:
        print("No optimal food combination found.")
else:
    print(f"User ID {input_user_id} not found.")



Optimal Food Combination:
Total Carbs: 45.17g (Target: 44.73g)
Total Protein: 36.98g (Target: 37.01g)
Total Fat: 23.21g (Target: 23.19g)

Foods in Combination:
- Bread, oat bran (Carbs: 39.8g, Protein: 10.4g, Fat: 4.4g)
- Fish, salmon, grilled (Carbs: 0g, Protein: 25.92g, Fat: 16.48g)
- Oat milk (Carbs: 5.37g, Protein: 0.66g, Fat: 2.33g)
